## Agenda


- Confidence intervals

    - Standard error
    - Confidence factors
    - Margin of Error  
  
- Constructing a confidence intervals for the bike share trip durations

- Z-scores for the normal distributions



## Confidence Intervals


A point estimate `x_bar`, of the mean, provides a single plausible value for a parameter. However, as we have seen, a point estimate is rarely perfect; usually there is some error in the estimate. That is why we have suggested using the standard error as a measure of its variability.

Instead of that, a next logical step would be to provide a __plausible range of values__ for the parameter. A plausible range of values for the sample parameter is called a __confidence interval.__

<img src = "./img/margin_of_error.png" width = 450 />

Point estimate +/- margin of error

**KEY POINT** : Our level of confidence that if we obtained a sample of equal size, our sample interval would contain the population mean.

 - This implies that there is an element of chance whether this interval will contain the true mean or not. In fact, when we calculate 95% confidence intervals, we should expect for every 20 samples and 20 confidence intervals created from these samples, on of them might miss the true parameter.

Let's understand this better using a [visual display](https://shiny.rit.albany.edu/stat/confidence/).

### Z-scores

In [ ]:
from scipy.stats import norm

In [ ]:
mu =90

sigma = 5

num_std = 3

In [ ]:
x_bar = 81

In [ ]:
z_score = (x_bar - mu)/sigma

In [ ]:
norm.pdf(0)

In [ ]:
x = np.linspace(mu - num_std*sigma, mu + num_std*sigma, 100)
y = norm.pdf(x , loc= mu, scale = sigma)

In [ ]:
norm.pdf(z_score)

In [ ]:

plt.figure(figsize = (10,8))
plt.subplot(1,2,1).plot(x,y)
plt.vlines(x =mu, ymin = 0, ymax = norm.pdf(mu, loc = mu, scale = sigma) )

plt.vlines(x = [mu - sigma, mu + sigma], ymin = [0,0 ], 
           ymax = [norm.pdf(mu-sigma, loc = mu, scale = sigma),norm.pdf(mu+sigma, loc = mu, scale = sigma)], color = 'red')

plt.vlines(x = x_bar, ymin = 0, ymax = norm.pdf(x_bar, loc = mu, scale = sigma), color = 'green')
plt.title("A Normal Distribution with mean %.2f and std %.2f"%(mu, sigma))
x_standard = np.linspace(-num_std, num_std, 100)
y_standard = norm.pdf(x_standard)

plt.subplot(1,2,2).plot(x_standard,y_standard)
plt.vlines(x =0, ymin = 0, ymax = norm.pdf(0, loc = 0, scale = 1) )

plt.vlines(x = [-1,1], ymin = [0.01, 0.01], 
           ymax = [norm.pdf(-1),norm.pdf(1)], color = 'red')

plt.vlines(x = z_score, ymin = 0, ymax = norm.pdf(z_score), color = 'green')
plt.title("Standardization of the Normal Distribution")
plt.show()

### An Example

Let's try to create a %95 confidence interval for the average Capital Bike Share trips durations.

For this problem we can assume that we know the population standard deviation $\sigma = 17$. Note that $\mu$ is not known and we are trying to estimate it.

[For more info about the data](https://www.capitalbikeshare.com/system-data)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('data/sampled_bikeshare.csv')
df.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
print(df.shape)
df.head()


In [ ]:
## convert seconds to mins
trip_durations = df['Duration']/60
trip_durations.describe()

Note that there are some outliers. Let's take a closer look at them.

In [ ]:
trip_durations.plot(kind = 'box')

In [ ]:

## Considering only the trips less than 2 hours
two_hours_trips = trip_durations[trip_durations < 120]

In [ ]:
two_hours_trips.hist()
plt.title("Distribution of ALL Samples Collected")
plt.xlabel("Duration in Min")
plt.ylabel("Frequency")
plt.show()

Note that distribution is definitely not normally distributed.

In [ ]:
## Find the point estimator for the population mean:

x_bar = two_hours_trips.mean()

print("In our sample the average trip length is: %.2f"%x_bar)

In [ ]:
## We can also calculate point estimators for the median and variance and std
## recall that the sample statistics for standard deviation is denoted with "s"
s = two_hours_trips.std()
print("Sample standard deviation is: %.2f"%s)

### Basic Principles in the Construction of Confidence Intervals

 - Our point estimate is the most plausible value of the parameter, so it makes sense to build the confidence interval around the point estimate.

 - The plausability of a range of values can be defined from the sampling distribution of the estimate.

### Central Limit Theorem Recap:


Given a population with a mean $\mu$ and a variance $\sigma^{2}$, the sampling distribution of the mean approaches a normal distribution with a mean of $\mu$ and standard deviation of $\sqrt{\frac{\sigma^{2}}{n}}$ as n, the sample size, increases.


__Note__

$\sqrt{\frac{\sigma^{2}}{n}}$ will be called the standard error of the mean. We usually denote it with `SE`. Note that standard error of the mean is actually standard deviation of the sampling distribution of the mean.

#### Central Limit Theorem with Respect to Different Population Distributions

In [ ]:
from scipy.stats import uniform

In [ ]:
from scipy.stats import expon

In [ ]:
from scipy.stats import poisson

In [ ]:
import math

In [ ]:
def central_limit_plot(dist_name, population_size, sample_size, num_samples):
    """
    This function plots the original population distribution and
    the sampling distribution of the mean derived from this population
    """
    if dist_name == "uniform":
        distribution =uniform.rvs(size = population_size)
        mu, sigma = uniform.stats(moments = 'mv')
        sampling_mean_distribution = []
        for i in range(num_samples):
            sample = uniform.rvs(size = sample_size)
            sampling_mean_distribution.append(sample.mean())
    if dist_name == "exponential":
        distribution =expon.rvs(size = population_size)
        mu, sigma = expon.stats(moments = 'mv')
        sampling_mean_distribution = []
        for i in range(num_samples):
            sample = expon.rvs(size = sample_size)
            sampling_mean_distribution.append(sample.mean())
    if dist_name == "poisson":
        distribution =poisson.rvs(mu =math.e, size = population_size)
        mu, sigma = poisson.stats(mu = math.e, moments = 'mv')
        sampling_mean_distribution = []
        for i in range(num_samples):
            sample = poisson.rvs(mu = math.e, size = sample_size)
            sampling_mean_distribution.append(sample.mean())
    sampling_mu = np.mean(sampling_mean_distribution)
    empirical_standard_error = np.std(sampling_mean_distribution)
    se = np.sqrt(sigma/sample_size)
    
    plt.figure(figsize = (10, 8))
    plt.subplot(1,2,1).hist(distribution)
    plt.title("%s Distribution: $\mu$ =%.2f std: %.2f"%(dist_name,mu,sigma))
    plt.subplot(1,2,2).hist(sampling_mean_distribution)
    plt.axvline(x = np.mean(sampling_mean_distribution), 
                color = 'yellow',
                label=  "$\mu$= %.2f"%sampling_mu)
    plt.axvline(x = mu - empirical_standard_error, color = 'red', linestyle = "--", label = "Emprical SE: %.2f"%empirical_standard_error)
    plt.axvline(x = mu + empirical_standard_error, color = 'red', linestyle = "--", label = "SE: %.2f"%se)
    plt.title("Sampling distribution of the mean")
    plt.legend()
    plt.show()

In [ ]:
central_limit_plot(dist_name = "uniform", population_size= 2000, sample_size = 100, num_samples= 1000)

Now we can easily construct a confidence interval for mean trip durations:

In [ ]:
x_bar = two_hours_trips.mean()

## Let's find sample size and record it as n

n= len(two_hours_trips)

## Recall that  sigma is known and 17

sigma = 17

## Standard Error:

se = np.sqrt(sigma**2/n)

print("Standard Error is %.2f"%se)

In [ ]:
## Confidence interval for the mean

CI = (x_bar - 1.96*se, x_bar + 1.96*se)

CI

In [ ]:
## Population mean (mu) is 17.32